In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors

from datetime import datetime

In [42]:
stop_events = pd.read_csv("../data_files/B/stop_events.csv")

stop_events.head()

stop_events = stop_events.dropna(axis=0)


In [43]:
stop_events["prev_actualArrival"] = stop_events["prev_actualArrival"].astype(
    "datetime64[ns]"
)
stop_events["prev_actualDeparture"] = stop_events["prev_actualDeparture"].astype(
    "datetime64[ns]"
)
stop_events["actualArrival"] = stop_events["actualArrival"].astype(
    "datetime64[ns]"
)
stop_events["actualDeparture"] = stop_events["actualDeparture"].astype(
    "datetime64[ns]"
)

In [44]:
stop_events["segment_duration"] = (
        stop_events.actualArrival - stop_events.prev_actualDeparture
    ).astype("timedelta64[s]")

In [45]:
stop_events.columns

Index(['id', 'date', 'stopCode', 'aimedArrival', 'aimedDeparture',
       'actualArrival', 'actualDeparture', 'vehicle', 'workid', 'patternId',
       'publicName', 'scheduledStart', 'prev_stopCode', 'prev_aimedArrival',
       'prev_aimedDeparture', 'prev_actualArrival', 'prev_actualDeparture',
       'timingPoint', 'prev_timingPoint', 'segment_code', 'segment_name',
       'segment_duration'],
      dtype='object')

In [54]:
means = stop_events.groupby("segment_code")['segment_duration'].mean()

medians = stop_events.groupby("segment_code")['segment_duration'].median()

In [47]:
std = stop_events.groupby("segment_code")['segment_duration'].std()

In [55]:
means = means.rename("means")
medians = medians.rename("medians")
std = std.rename("std")

In [49]:
stop_events = stop_events.merge(means.to_frame(), left_on="segment_code", right_index=True)

In [56]:
stop_events = stop_events.merge(medians.to_frame(), left_on="segment_code", right_index=True)

In [50]:
stop_events = stop_events.merge(std.to_frame(), left_on="segment_code", right_index=True)

In [51]:
stop_events.columns

Index(['id', 'date', 'stopCode', 'aimedArrival', 'aimedDeparture',
       'actualArrival', 'actualDeparture', 'vehicle', 'workid', 'patternId',
       'publicName', 'scheduledStart', 'prev_stopCode', 'prev_aimedArrival',
       'prev_aimedDeparture', 'prev_actualArrival', 'prev_actualDeparture',
       'timingPoint', 'prev_timingPoint', 'segment_code', 'segment_name',
       'segment_duration', 'means', 'std'],
      dtype='object')

In [57]:
stop_events['outliers'] = False
stop_events.loc[(stop_events['segment_duration'] > (stop_events['medians'] + 3 * stop_events['std'])) | (stop_events['segment_duration'] < (stop_events['medians'] - 3 * stop_events['std'])), 'outliers'] = True

# stop_events.loc[(stop_events['segment_duration'] > (stop_events['means'] + 5 * stop_events['std'])), 'outliers'] = True
                                                                                                       

In [58]:
np.count_nonzero(stop_events['outliers'])

184556

In [35]:
segment_counts = stop_events.groupby("segment_code").size()

In [36]:
stop_events['minor_segment'] = False
stop_events.loc[
            stop_events["segment_code"].isin(
                segment_counts[segment_counts < 120].index.values
            )
        , "minor_segment"] = True

In [37]:
np.count_nonzero(stop_events['minor_segment'])

16325

In [38]:
stop_events = stop_events.dropna(axis=0)

In [39]:
dropped_se.shape

(4256701, 26)

In [40]:
stop_events.shape

(4604714, 26)

In [41]:
stop_events.shape[0] - dropped_se.shape[0]

348013